In [ ]:
import arcpy as ap
from arcpy import env

workspace = r'C:\Users\broadways\CDOT_temp\Arc\Modeling_Network'

In [ ]:
# Creating a spatial reference object; GRS 1980
sr = arcpy.SpatialReference(4019)
ap.env.outputCoordinateSystem = sr

In [ ]:
import arcpy as ap
import os


# create variables for directories that will be used throughout this notebook
local_directory = r'\\CDOTDBD01\Shared\Conflating_Networks'
directory_containing_original_shapefiles = local_directory + '\\network_shapefiles\Test'
local_geodatabase = local_directory + '\\working_files.gdb'


# define the workspace as the location of the original shapefiles and create a list of the shapefiles
ap.env.workspace = directory_containing_original_shapefiles
network_shapefiles = [x[:-4] for x in ap.ListFeatureClasses()]
ap.env.workspace = local_directory

# if the geodatabase doesn't exist, create it
if ap.Exists(local_geodatabase) == False:
    ap.CreateFileGDB_management(local_directory, 'working_files.gdb')
    
# define spatial reference as NAD_1983_2011_UTM_Zone_13N
spatial_reference = ap.SpatialReference(102382)
ap.env.outputCoordinateSystem = spatial_reference

ap.env.workspace = local_geodatabase

In [ ]:
# Create geodatabase to hold the feature dataset
env.workspace = r"C:\Users\broadways\CDOT_temp\Arc\Modeling_Network"

geodatabase = "\Modeling_Network.gdb"

ap.CreateFileGDB_management(workspace, geodatabase)


In [ ]:
# Create feature dataset

# CreateFeatureDataset_management (out_dataset_path, out_name, {spatial_reference})

out_dataset_path = workspace + "\Modeling_Network.gdb"
out_name = "network"

arcpy.CreateFeatureDataset_management(out_dataset_path, out_name, sr)

In [ ]:
# Convert shapefile (input data) to feature class and import into geodatabase feature dataset

# FeatureClassToFeatureClass_conversion (in_features, out_path, out_name, {where_clause}, {field_mapping}, {config_keyword})

in_features = "DTD_2010_links.shp"
out_path = r"C:\Users\broadways\CDOT_temp\Arc\Modeling_Network\Modeling_Network.gdb\network"
out_name = "network_links"

arcpy.FeatureClassToFeatureClass_conversion(in_features, out_path, out_name)


In [ ]:
# Create topology for feature dataset

# CreateTopology_management (in_dataset, out_name, {in_cluster_tolerance})

in_dataset = r"C:\Users\broadways\CDOT_temp\Arc\Modeling_Network\Modeling_Network.gdb\network"
out_name = "network_Topology"

ap.CreateTopology_management(in_dataset, out_name)


In [ ]:
# Name: AddFeatureClassToTopology_Example.py
# Description: Adds a feature class to participate in a topology


# AddFeatureClassToTopology_management (in_topology, in_featureclass, xy_rank, z_rank)

in_topology = r"C:\Users\broadways\CDOT_temp\Arc\Modeling_Network\Modeling_Network.gdb\network\network_Topology"
in_featureclass = r"C:\Users\broadways\CDOT_temp\Arc\Modeling_Network\Modeling_Network.gdb\network\network_links"

#arcpy.AddFeatureClassToTopology_management(in_topology, in_featureclass)


In [ ]:
# Name: AddRuleToTopology_Example.py
# Description: Adds a rule to a topology

# AddRuleToTopology_management (in_topology, rule_type, in_featureclass, {subtype}, {in_featureclass2}, {subtype2})


# Any intersection of ParcelOutline (BlockLines subtype only) needs to be reviewed
arcpy.AddRuleToTopology_management(in_topology, 
                                   "Must Not Have Dangles (Line)",
                                   in_featureclass)
arcpy.AddRuleToTopology_management(in_topology,
                                   "Must Not Self-Overlap (Line)",
                                   in_featureclass)
arcpy.AddRuleToTopology_management(in_topology,
                                   "Must Be Single Part (Line)",
                                   in_featureclass)

In [ ]:
# Name: ValidateTopology_Example.py
# Description: Validates a topology

# ValidateTopology_management (in_topology, {visible_extent})

arcpy.ValidateTopology_management(in_topology)

In [ ]:
# Set the necessary product code
import arceditor

# ExportTopologyErrors_management (in_topology, out_path, out_basename)

# Process: Export Topology Errors
arcpy.ExportTopologyErrors_management(in_topology, out_dataset_path, "errors_network_topology")


In [ ]:
## Possible topology rules:
    Must Not Have Gaps (Area) | Must Not Overlap (Line) | Must Not Intersect (Line) | Must Not Have Dangles (Line) | 
    Must Not Have Pseudo-Nodes (Line) | Must Be Covered By Feature Class Of (Line-Line) | 
    Must Not Overlap With (Line-Line) | Must Be Covered By (Point-Line) | Must Be Covered By Endpoint Of (Point-Line) | 
    Boundary Must Be Covered By (Area-Line) | Boundary Must Be Covered By Boundary Of (Area-Area) | 
    Must Not Self-Overlap (Line) | Must Not Self-Intersect (Line) | Must Not Intersect Or Touch Interior (Line) | 
    Endpoint Must Be Covered By (Line-Point) | Contains Point (Area-Point) | Must Be Single Part (Line) | 
    Must Coincide With (Point-Point) | Must Be Disjoint (Point) | Must Not Intersect With (Line-Line) | 
    Must Not Intersect or Touch Interior With (Line-Line) | Must Be Inside (Line-Area) | Contains One Point (Area-Point).

